In [6]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [7]:
from dpo_utils import *
from dpo_data_module import CombinedForgetRetainDataset
from collators import dpo_retain_collator
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from accelerate import  Accelerator
from config import Config
import torch
from peft import  LoraConfig, get_peft_model
from utils import find_all_linear_names
import pandas as pd
from torch.utils.data import Subset

In [8]:
cfg = Config()

accelerator = Accelerator()

In [9]:
cfg.save_dir = 'outputs/wpu_batch_npo_2_6'
cfg.save_dir

'outputs/wpu_batch_npo_2_6'

In [10]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct', token = cfg.access_token)
if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

In [11]:
policy_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_id,
    torch_dtype=torch.bfloat16, 
    token=cfg.access_token 
    )
print("Base model loaded.")


# --- Apply LoRA on policy model ---
print("Applying LoRA...")
lora_config = LoraConfig(
    r=cfg.LoRA_r,
    lora_alpha=cfg.LoRA_alpha,
    lora_dropout=cfg.LoRA_dropout,
    target_modules=find_all_linear_names(policy_model), 
    bias='none',
    task_type='CAUSAL_LM',
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Base model loaded.
Applying LoRA...


In [12]:
# Get PEFT model 
model = get_peft_model(policy_model, lora_config)
print("PEFT model created.")
model.print_trainable_parameters()
model.config.use_cache = False # Important for gradient checkpointing


PEFT model created.
trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605


In [13]:
ref_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_id,
    torch_dtype=torch.bfloat16,
    token=cfg.access_token
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
forget = pd.read_csv(cfg.forget_path)
retain = pd.read_csv(cfg.retain_path)

In [15]:
forget['factor'] = -1.0
retain['factor'] = 1.0
forget['factor'] = forget['factor'].astype('float')
retain['factor'] = retain['factor'].astype('float')
retain['idk'] = 'idk'


In [16]:
total_batch_size = 8
n_forget_in_batch = 2
n_retain_in_batch = total_batch_size - n_forget_in_batch
print(f"Batch size: {total_batch_size}, Forget samples in batch: {n_forget_in_batch}, Retain samples in batch: {n_retain_in_batch}")

Batch size: 8, Forget samples in batch: 2, Retain samples in batch: 6


In [17]:
train_dataset =  CombinedForgetRetainDataset(
    forget_df = forget,
    retain_df = retain,
    tokenizer = tokenizer,
    max_length = 256,
    block_size = total_batch_size,
    n_forget   = n_forget_in_batch,
    n_retain   = n_retain_in_batch
)


Combined dataset initialized with 2432 samples.
Verifying sample structure (first few blocks):
  Block 0: 2 forget, 6 retain samples. Expected: 2, 6
  Block 1: 2 forget, 6 retain samples. Expected: 2, 6
  Block 2: 2 forget, 6 retain samples. Expected: 2, 6


In [18]:
training_args = TrainingArguments(
        output_dir = f'{cfg.save_dir}',
        overwrite_output_dir= True,
        max_grad_norm=1.0,
        learning_rate = cfg.lr,
        per_device_train_batch_size= cfg.batch_size, 
        num_train_epochs= cfg.num_epochs,
        weight_decay = cfg.weight_decay,
        logging_dir = f'{cfg.save_dir}/logs',
        logging_steps= 250,
        eval_strategy= 'no',
        label_names = ['labels'],
        bf16 = True,
        gradient_accumulation_steps= cfg.gradient_accumulation_steps,
        remove_unused_columns=False,
        report_to = 'wandb',
        seed = 42,
        ddp_find_unused_parameters=False,
)

In [19]:
trainer = BatchRetainNPOTrainer(
      model = model,
      ref_model= ref_model,
      args = training_args,
      train_dataset = train_dataset, 
      data_collator = dpo_retain_collator,
      beta=cfg.npo_beta,
)

[2025-05-18 12:45:14,765] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: /usr/local/cuda-12/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/praveen/miniconda3/envs/emnlp/compiler_compat/ld: /usr/local/cuda-12/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXX

Preparing reference model...
Rank 0: In _prepare_ref_model. Accelerator device: cuda, torch.cuda.current_device(): 0
Reference model prepared and set to eval mode.


In [20]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Rank 0: Instantiating SequentialSampler for single GPU.


wandb: Currently logged in as: praveenbushipaka942 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
250,21.933600
500,12.834300
750,10.602500
1000,9.161200
1250,8.012700
1500,7.213500
1750,6.285800
2000,5.957200
2250,5.589300
2500,5.091200


TrainOutput(global_step=3040, training_loss=8.499847296664589, metrics={'train_runtime': 4493.1333, 'train_samples_per_second': 5.413, 'train_steps_per_second': 0.677, 'total_flos': 0.0, 'train_loss': 8.499847296664589, 'epoch': 10.0})

In [21]:
model.save_pretrained(cfg.save_dir)
tokenizer.save_pretrained(cfg.save_dir)

('outputs/wpu_batch_npo_2_6/tokenizer_config.json',
 'outputs/wpu_batch_npo_2_6/special_tokens_map.json',
 'outputs/wpu_batch_npo_2_6/tokenizer.json')